In [ ]:
# importing the requests library
import requests
import json

# api-endpoint
# URL_GET = "https://erisk.irlab.org/challenge-service/getwritings/%s"
URL_GET = "https://erisk.irlab.org/challenge-t1/getwritings/%s"
# URL_POST = "https://erisk.irlab.org/challenge-service/submit/%s/%d"
URL_POST = "https://erisk.irlab.org/challenge-t1/submit/%s/%d"
TOKEN = "q95QLQYOeqpMuwrRdggKZ1F614619WSGS9TEyQl2bZ4"



In [ ]:
def get_users():

    # defining a params dict for the parameters to be sent to the API
    PARAMS = {}

    # sending get request and saving the response as response object
    r = requests.get(url=URL_GET%TOKEN, params=PARAMS)

    # extracting data in json format
    data = r.json()

    return data



In [ ]:
def process_data(json_data):
    subjects = [d['nick'] for d in json_data]
    return subjects

def get_subjects(filepath):
    subjects = []
    with open(filepath) as f:
        for line in f:
            subjects.append(json.loads(line)['nick'])
    return subjects



In [ ]:
def serialize_data(data, filepath_data=None, filepath_subjects=None):
    rnd = data[0]['number']
    if not filepath_data:
        filepath_data = 'data_server/data%d.jl'%rnd
    if not filepath_subjects:
        filepath_subjects = 'data_server/subjects%d.txt'%rnd
    with open(filepath_data, 'w+') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')
    with open(filepath_subjects, 'w+') as f:
        for item in process_data(data):
            f.write(item + '\n')

def serialize_response(response, rnd, run):
    with open('data_server/response_rnd%d_run%d.jl'%(rnd, run), 'w+') as f:
        f.write(response)

In [ ]:
def send_prediction(run_nr, rnd, predictions):
    data = []
    for subject, score, label in predictions:
        data.append({
            'nick': subject,
            'decision': label,
            'score': score
        })
    print('prediction len', len(data))
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    response = requests.post(url=URL_POST%(TOKEN,run_nr), data=json.dumps(data), headers=headers)
    serialize_response(response, rnd=rnd, run=run_nr)
    return response

def send_response(response, run_nr, rnd):
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    response_server = requests.post(url=URL_POST%(TOKEN,run_nr), data=json.dumps(response), headers=headers)
#     serialize_response(response_server, rnd, run_nr)
    return response_server

In [ ]:
def get_predictions_dummy(data, predictions, scores):
    data = process_data(data)
    subjects = get_subjects()
    # dummy predictions
    predictions = [(s, predictions[s], scores[s]) for s in subjects]
    return predictions

def get_response_from_file(run, rnd):
    predictions_json = read_data('data_server/response_run%d_rnd%d.json' % (run, rnd))
    return predictions_json


def read_data(filepath):
    data = []
    with open(filepath) as f:
        for line in f:
            data.append(json.loads(line))
    return data



In [ ]:
data = get_users()

print('len data', len(data), data)
serialize_data(data)




In [ ]:
    #for run in range(5):

     #   print('run', run)
      #  predictions = get_predictions(data)
        #print(send_prediction(run, predictions))
        # You get new round once you submit your results for run 5.
        # What if you submit for run 5 in the beginning tho? Still doesn't give you the new ones right after run 5. It waits to get all of them that's it.
        # You should verify when building data for a new round that you are on the right round... that's it.
        # But it seems to be working properly.
       # print(get_users())

In [ ]:
json.dumps(predictions[0])

In [ ]:
%%time

for i in range(101):
    rnd = 599
    # assert rnd==data[0]['number']
    for run_nr in [0,1,2,3,4]:
        print("run", run_nr, "sending round", rnd)
        try:
            predictions = get_response_from_file(run=run_nr, rnd=rnd)
                
        except Exception:
            print("Round %d not available for run %d" % (rnd, run_nr))
            try:
                predictions =  get_response_from_file(run=run_nr, rnd=20)
                print("run", run_nr, "sending round", 20)
            except Exception:
                print("Round %d not available for run %d" % (20, run_nr))
                predictions =  get_response_from_file(run=run_nr, rnd=1)
                print("run", run_nr, "sending round", 1)
        response_server = send_response(predictions[0], run_nr=run_nr, rnd=rnd)
    data = get_users()

    print('len data', len(data), "getting rnd", data[0]['number'])
    serialize_data(data)

In [ ]:
%%time
data = get_users()

print('len data', len(data), "getting rnd", data[0]['number'])
serialize_data(data)

In [ ]:
response_server

In [ ]:
# Start round
rnd = -1
# Stop round
rnd_thresh = 50
# Submit frequency
rnd_batch_size = 10

while True:
    # Get data
    data = get_users()
    serialize_data(data)
    cur_round = data['number']
    print('len data', len(data), cur_round)
    if cur_round == rnd:
        break
    rnd = cur_round
    # TO remove
    if rnd >= rnd_thresh:
        break
    
    if rnd % rnd_batch_size == 0:
        print("Seding results (round %d...)" % rnd)
        for run in range(5):
            predictions = get_response_from_file(model=models[run], run=run, rnd=rnd)
            response_server = send_response(predictions, run_nr=run, rnd=run)
            print(response_server)
        
    
    